In [4]:
from scipy.integrate import dblquad

def f(x, y):
    return x * y

def g(x, y):
    return x + y

def w(x, y):
    return f(x, y) + g(x, y)

a, b = 0, 1
c, d = 0, 1

result, error = dblquad(w, a, b, c, d)
print(result)

1.25


In [9]:
import numpy as np

a = np.array([1,2])
h = lambda x : x + 1
print(h(a))

[2 3]


In [128]:
import time 
import numpy as np
from numpy.linalg import solve
from scipy.sparse.linalg import spsolve
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt

from fealpy.decorator import cartesian
from fealpy.mesh import TriangleMesh
from fealpy.functionspace import LagrangeFiniteElementSpace
from fealpy.boundarycondition import DirichletBC

class PDE():
    def __init__(self, mu=1, lam=1):
        self.mu  = mu
        self.lam = lam
        
    def domain(self):
        return [0, 1, 0, 1]
    
    def init_mesh(self, n=1, meshtype='tri'):
        node = np.array([
            (0,0),
            (1,0),
            (1,1),
            (0,1)], dtype=np.float64)
        cell = np.array([(1,2,0), (3,0,2)], dtype=np.int64)
        mesh = TriangleMesh(node, cell)
        mesh.uniform_refine(n)
        return mesh

#判断 P (维度[2]) 是否在 cr_node, 是则放回其下标，否则 (val = 1 时) 将 P 加入 cr_node 并返回下标
def is_in_cr_node(p, cr_node, val=0):
    #p 不会为[0,0]
    #print("p= ", p)
    #print("cr_node[0]= ", cr_node[0])
    index = np.where((cr_node == p).all(axis=1))[0]
    if len(index):
        return index[0]
    else:
        in_index = np.where((cr_node == np.array([0,0])).all(axis=1))[0]
        if len(in_index) == 0:
            print("len(in_index)= ", len(in_index))
            raise Exception("数组cr_node已满")
        cr_node[in_index[0]] = p
        return in_index[0]

#将 a_cell (维数[3]) 放入new_cr_cell
def is_in_cr_cell(a_cell, new_cr_cell):
    #for i in range(new_cr_cell.shape[0]):
    #    if (new_cr_cell[i] == [0,0,0]).all():
    #        new_cr_cell[i] = a_cell
    #        return new_cr_cell
    #print("a_cell= ", a_cell)
    #print("new_cr_cell= ", new_cr_cell)
    in_index = np.where((new_cr_cell == np.array([0,0,0])).all(axis=1))[0]
    #print("in_index= ", in_index)
    if len(in_index) == 0:
        #print("in_index= ", in_index)
        raise Exception("数组cr_cell已满")
    new_cr_cell[in_index[0]] = a_cell
    return new_cr_cell
        

# 对单个三角形 a_cell_node (维度 [3, 2]) 求三条边中点 p1, p2, p3 并将其放入 new_cr_node 、 new_cr_cell
def a_creat(a_cell_node, new_cr_node, new_cr_cell):
    #print("a_cell_node= ", a_cell_node)
    p1 = (a_cell_node[0] + a_cell_node[1]) / 2
    p2 = (a_cell_node[0] + a_cell_node[2]) / 2
    p3 = (a_cell_node[1] + a_cell_node[2]) / 2
    
    p1_i = is_in_cr_node(p1, new_cr_node, val=1)
    p2_i = is_in_cr_node(p2, new_cr_node, val=1)
    p3_i = is_in_cr_node(p3, new_cr_node, val=1)
    
    is_in_cr_cell([p1_i, p2_i, p3_i], new_cr_cell)
    
    return new_cr_node, new_cr_cell
    

# 由端点的 node 、cell 得到边中点的 cr_node cr_cell
def get_cr_node_cell(n, cr_node, cr_cell, node, cell):
    old_NN = cr_node.shape[0]
    old_NC = cr_cell.shape[0]
    
    mesh_text = TriangleMesh(node, cell)
    mesh_text.uniform_refine(n)
    #[NN,2] 剖分点及其编号(下标)
    node = mesh_text.entity('node')
    #print("node= ", node)
    #[NC,3] 剖分区间及其端点编号
    cell = mesh_text.entity('cell')
    NN = mesh_text.number_of_faces()
    NC = mesh_text.number_of_cells()
    #print("old_NN= ", old_NN)
    #print("NN= ", NN)
    print("NC= ", NC)
    #print("cell.shape= ", cell.shape)
    
    new_cr_node = np.zeros((NN, 2), dtype=np.float64)
    new_cr_cell = np.zeros((NC, 3), dtype=np.int64)
    
    if n == 0:
        new_cr_node[0:old_NN, :] = cr_node
        new_cr_cell[0:old_NC, :] = cr_cell
        return new_cr_node, new_cr_cell
    for i in range(NC):
        #print(i)
        #print("cell= ", cell)
        new_cr_node, new_cr_cell = a_creat(node[cell[i]], new_cr_node, new_cr_cell)
    
    print("node_cell_e= ", np.max(np.abs(node[cell] - new_cr_node[new_cr_cell])))
    
    return new_cr_node, new_cr_cell

# 用于求解 cr_node, cr_cell
old_node = np.array([
    (0,0),
    (1,0),
    (1,1),
    (0,1)
], dtype=np.float64)
old_cell = np.array([
    (1,2,0),
    (3,0,2)
], dtype=np.int64)

cr_node = np.array([
            (0.5,0),
            (1,0.5),
            (0.5,0.5),
            (0,0.5),
            (0.5,1)
        ], dtype=np.float64)

cr_cell = np.array([
        (0,1,2),
        (2,3,4)
    ], dtype=np.int64)

n =6
start = time.time()
cr_node, cr_cell = get_cr_node_cell(n, cr_node, cr_cell, old_node, old_cell)
end = time.time()
print("运行时间 : ", end-start)
#print("cr_node= ", cr_node)
#print("cr_cell= ", cr_cell)

NC=  8192
node_cell_e=  0.015625
运行时间 :  10.131927013397217


In [84]:
import numpy as np

a = np.array([0,0])
b = np.array([[2,1],
             [0,0],
             [3,4],
             [0,0]])

c = np.array([0])
d = np.array([1,2,0,3,4])

print(np.isin(a,b))
print(np.where((b == a).all(axis=1))[0])

[ True  True]
[1 3]


In [107]:
import numpy as np

a = np.array([1,0,0])
b = np.array([[2,3,5],
            [1,2,3],
            [1,0,0],
            [0,0,0],
            [0,0,0]])

print(np.where((b == a).all(axis=1))[0])

[2]
